# Solución Caso 1

## Conjuntos Necesarios

In [10]:
import pandas as pd
#Clientes y sus datos
clients_df = pd.read_csv('Clients.csv')
clients_dict = {f"C{int(row['ClientID'])}": [int(row['Product']),row['Longitude'],row['Latitude']] for _, row in clients_df.iterrows()}
print(clients_dict)

#Depositos y sus datos
deposits_df = pd.read_csv('Depots.csv')
# deposits_df = pd.read_csv('single_depot.csv')
deposits_dict = {f"D{int(row['DepotID'])}": [int(row['DepotID']),row['Longitude'],row['Latitude']] for _, row in deposits_df.iterrows()}
print(deposits_dict)

#Vehículos y sus datos
vehicles_df = pd.read_csv('Vehicles.csv')
type_mapping = {
    'gas car': 'GS',
    'ev': 'EV',
    'solar ev': 'EV',
    'drone': 'DR'
}
vehicles_dict = {}
for idx, row in enumerate(vehicles_df.itertuples(), start=1):
    vehicle_id = f"V{idx}"  # Claves como V1, V2, ...
    vehicles_dict[vehicle_id] = [type_mapping[row.VehicleType.lower()], row.Capacity, row.Range]
print(vehicles_dict)

#Datos de Costos por tipo de vehículo
vehicles_data_df = pd.read_csv('vehicles_data.csv')
vehicles_data_df.rename(
    columns={
        'Vehicle': 'Type',
        'Freight Rate [COP/km]': 'TF',
        'Time Rate [COP/min]': 'TT',
        'Daily Maintenance [COP/day]': 'CM',
        'Recharge/Fuel Cost [COP/(gal or kWh)]': 'CRC',
        'Recharge/Fuel Time [min/10 percent charge]': 'TR',
        'Avg. Speed [km/h]': 'VP',
        'Gas Efficiency [km/gal]': 'Gas_Efficiency',
        'Electricity Efficency [kWh/km]': 'Electricity_Efficiency'
    },
    inplace=True
)

vehicles_parameters = {}
for _, row in vehicles_data_df.iterrows():
    vehicle_type = type_mapping[row['Type'].lower()]
    vehicles_parameters[vehicle_type] = {
        'TF': row['TF'],
        'TT': row['TT'],
        'CM': row['CM'],
        'CRC': row['CRC'],
        'TR': row['TR'],
        'VP': row['VP'],
        'EC': row['Gas_Efficiency'] if not pd.isna(row['Gas_Efficiency']) else row['Electricity_Efficiency']
    }
print(vehicles_parameters)

#Matriz de distancias terrestres
distance_matrix_df = pd.read_csv('distance_matrix.csv', index_col=0)
# distance_matrix_df = pd.read_csv('distance_single_matrix.csv', index_col=0)
distance_dict = {
    (row, col): float(distance_matrix_df.loc[row, col])
    for row in distance_matrix_df.index
    for col in distance_matrix_df.columns
}
print({key: distance_dict[key] for key in list(distance_dict.keys())[:20]})

#Matriz de tiempos terrestres
duration_matrix_df = pd.read_csv('duration_matrix.csv', index_col=0)
# duration_matrix_df = pd.read_csv('duration_single_matrix.csv', index_col=0)
duration_dict = {
    (row, col): float(duration_matrix_df.loc[row, col])
    for row in duration_matrix_df.index
    for col in duration_matrix_df.columns
}
print({key: duration_dict[key] for key in list(duration_dict.keys())[:20]})

#Matriz de distancias aéreas
distance_air_df = pd.read_csv('distance_dron_matrix.csv', index_col=0)
# distance_air_df = pd.read_csv('distance_dron_single_matrix.csv', index_col=0)
distance_air_dict = {
    (row, col): float(distance_air_df.loc[row, col])
    for row in distance_air_df.index
    for col in distance_air_df.columns
}
print({key: distance_air_dict[key] for key in list(distance_air_dict.keys())[:20]})

{'C1': [13, -74.09893796560621, 4.59795431125545], 'C2': [15, -74.07557103763986, 4.687820646838871], 'C3': [12, -74.10708524062704, 4.70949446000624], 'C4': [15, -74.09727965657427, 4.605029068682624], 'C5': [20, -74.16464148202755, 4.648463876533332], 'C6': [17, -74.12083799988112, 4.662137416953968], 'C7': [17, -74.02213076607309, 4.697499030379109], 'C8': [20, -74.17207549744595, 4.649416884236942], 'C9': [20, -74.15615257246444, 4.606310650273935], 'C10': [15, -74.09041145358674, 4.557379705282216], 'C11': [17, -74.17802255204528, 4.591594072172954], 'C12': [12, -74.1015410917749, 4.7564172406324055], 'C13': [21, -74.09690889182339, 4.646217006050524], 'C14': [15, -74.1219200708342, 4.725912125314368], 'C15': [17, -74.0942948461378, 4.604168478560718], 'C16': [10, -74.11138839002187, 4.557320898243896], 'C17': [25, -74.12463941285208, 4.615869066082658], 'C18': [12, -74.12456164551857, 4.656402930181292], 'C19': [11, -74.04990580408855, 4.706188309535041], 'C20': [15, -74.12186680

## Modelo

In [11]:
from pyomo.environ import *

model = ConcreteModel()

# Conjuntos
unique_nodes = set(i for i, j in distance_dict.keys()).union(j for i, j in distance_dict.keys())
model.I = Set(initialize=unique_nodes)  # Nodos (orígenes y destinos)

model.PAIRS = Set(dimen=2, initialize=distance_dict.keys())  # Pares válidos (i, j)

model.K = Set(initialize=vehicles_dict.keys())  # Vehículos

# Parámetros

# Distancia y tiempo entre nodos
model.distance = Param(model.PAIRS, initialize=distance_dict)
model.duration = Param(model.PAIRS, initialize=duration_dict)
model.distance_air = Param(model.PAIRS, initialize=distance_air_dict)

# Parámetros del vehículo
model.TF = Param(model.K, initialize={k: vehicles_parameters[vehicles_dict[k][0]]['TF'] for k in model.K})
model.TT = Param(model.K, initialize={k: vehicles_parameters[vehicles_dict[k][0]]['TT'] for k in model.K})
model.CM = Param(model.K, initialize={k: vehicles_parameters[vehicles_dict[k][0]]['CM'] for k in model.K})
model.CRC = Param(model.K, initialize={k: vehicles_parameters[vehicles_dict[k][0]]['CRC'] for k in model.K})
model.TR = Param(model.K, initialize={k: vehicles_parameters[vehicles_dict[k][0]]['TR'] for k in model.K})
model.VP = Param(model.K, initialize={k: vehicles_parameters[vehicles_dict[k][0]]['VP'] for k in model.K})
model.EC = Param(model.K, initialize={k: vehicles_parameters[vehicles_dict[k][0]]['EC'] for k in model.K})
model.R = Param(model.K, initialize={k: vehicles_dict[k][2] for k in model.K})  # Rango de cada vehículo


# Variables de decisión
model.x = Var(model.I, model.I, model.K, domain=Binary)  # Si el vehículo k viaja de i a j
model.W = Var(model.K, domain=NonNegativeReals)  # Peso total transportado por vehículo k
model.u = Var(model.I, model.K, domain=NonNegativeReals)  # Posición secuencial en la ruta para evitar subtours


# Función objetivo
def total_cost_rule(model):
    # Costo por distancia (calculado desde los recorridos)
    C_total = sum(
        model.TF[k] * (
            model.distance[i, j]/1000 if vehicles_dict[k][0] != 'DR' else model.distance_air[i, j]
        ) * model.x[i, j, k]
        for k in model.K for i in model.I for j in model.I if i != j
    )
    
    # Costo por tiempo
    C_tiempo = sum(
        model.TT[k] * (
            model.distance[i, j]/1000  if vehicles_dict[k][0] != 'DR' else model.distance_air[i, j]
        ) * model.x[i, j, k] / model.VP[k]
        for k in model.K for i in model.I for j in model.I if i != j
    )
    
    # Costo por peso transportado
    C_carga = sum(model.W[k] * 500 for k in model.K)
    
    # Costo de mantenimiento diario
    C_mantenimiento = sum(model.CM[k] for k in model.K)

    # Costo total
    return C_total + C_tiempo + C_carga + C_mantenimiento
model.obj = Objective(rule=total_cost_rule, sense=minimize)

# Restricciones
# Restricción: Salida desde un depósito
def salida_desde_deposito_rule(model, k):
    return sum(model.x[d, j, k] for d in deposits_dict.keys() for j in model.I if d != j) >= 0
model.salida_desde_deposito = Constraint(model.K, rule=salida_desde_deposito_rule)

# Restricción: Regreso al depósito de origen
def regreso_a_deposito_rule(model, k):
    return sum(model.x[j, d, k] for d in deposits_dict.keys() for j in model.I if d != j) >= 0
model.regreso_a_deposito = Constraint(model.K, rule=regreso_a_deposito_rule)

def regreso_al_mismo_deposito_rule(model, k, d):
    if d in deposits_dict.keys():
        # Si el vehículo sale del depósito `d`, debe regresar a él
        return sum(model.x[d, j, k] for j in model.I if d != j) == sum(model.x[j, d, k] for j in model.I if d != j)
    else:
        return Constraint.Skip  # Ignorar para nodos que no son depósitos
# model.regreso_al_mismo_deposito = Constraint(model.K, deposits_dict.keys(), rule=regreso_al_mismo_deposito_rule)

# Restricción: Flujo correcto entre nodos (clientes y depósitos)
def flujo_correcto_rule(model, i, k):
    if i in deposits_dict.keys():  # No aplica para depósitos
        return Constraint.Skip
    return sum(model.x[i, j, k] for j in model.I if i != j) == sum(model.x[j, i, k] for j in model.I if i != j)
model.flujo_correcto = Constraint(model.I, model.K, rule=flujo_correcto_rule)


# Restricción: MTZ para evitar subtours
def mtz_rule(model, i, j, k):
    if i == j or i in deposits_dict.keys() or j in deposits_dict.keys():
        return Constraint.Skip
    n = len(model.I)
    return model.u[i, k] - model.u[j, k] + (n - 1) * model.x[i, j, k] <= n - 2
model.mtz = Constraint(model.I, model.I, model.K, rule=mtz_rule)

# Restricción: Capacidad Máxima de Carga
def capacidad_carga_rule(model, k):
    return model.W[k] <= vehicles_dict[k][1]  # Capacidad máxima del vehículo k
model.capacidad_carga = Constraint(model.K, rule=capacidad_carga_rule)

# Restricción: Demanda de los clientes
def demanda_clientes_rule(model, i):
    if i not in clients_dict.keys():
        return Constraint.Skip
    return sum(model.x[j, i, k] for k in model.K for j in model.I if j != i) == 1
model.demanda_clientes = Constraint(model.I, rule=demanda_clientes_rule)

# Restricción: Capacidad y Demanda
def capacidad_demanda_rule(model, k):
    return model.W[k] == sum(clients_dict[i][0] * sum(model.x[i, j, k] for j in model.I if i != j) for i in clients_dict.keys())
model.capacidad_demanda = Constraint(model.K, rule=capacidad_demanda_rule)

def distancia_maxima_rule(model, k):
    # Seleccionar la matriz de distancias según el tipo de vehículo
    return sum(
        (model.distance[i, j]/1000 if vehicles_dict[k][0] != 'DR' else model.distance_air[i, j]) * model.x[i, j, k]
        for i in model.I for j in model.I if i != j
    ) <= vehicles_dict[k][2]  # Rango máximo del vehículo
model.distancia_maxima = Constraint(model.K, rule=distancia_maxima_rule)


(type: set).  This WILL potentially lead to nondeterministic behavior in Pyomo


In [12]:
solver_name = "appsi_highs"
solver = SolverFactory(solver_name)
solver.options['parallel'] = 'on'
solver.options['threads'] = 6
solver.options['time_limit'] = 600  # 1-hour time limit
solver.options['mip_rel_gap'] = 0.05  # 5% relative gap
result = solver.solve(model, tee=True)

Running HiGHS 1.8.1 (git hash: 4a7f24a): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [3e-01, 4e+01]
  Cost   [2e+02, 2e+05]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 1e+03]
Presolving model
6960 rows, 12542 cols, 67941 nonzeros  0s
6622 rows, 9114 cols, 55081 nonzeros  2s
6404 rows, 8523 cols, 51640 nonzeros  6s

Solving MIP model with:
   6404 rows
   8523 cols (8223 binary, 0 integer, 12 implied int., 288 continuous)
   51640 nonzeros
MIP-Timing:         6.9 - starting analytic centre calculation

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSo

In [13]:
results_path = "results.txt"

with open(results_path, "w") as file:
    # Escribir el costo total
    file.write("=== Resultados ===\n")
    file.write(f"Costo total operativo: {model.obj():.2f} COP\n\n")

    # Escribir los recorridos por vehículo
    file.write("Recorridos asignados por vehículo:\n")
    for k in model.K:
        file.write(f"\nVehículo {k}:\n")
        recorrido = []
        for i in model.I:
            for j in model.I:
                if i != j and model.x[i, j, k].value > 0.5:  # Verifica si el vehículo viaja de i a j
                    recorrido.append((i, j))
        if recorrido:
            file.write(f"  Recorrido: {recorrido}\n")
        else:
            file.write("  No tiene asignaciones.\n")

    # Escribir distancias totales recorridas
    file.write("\nDistancias totales recorridas:\n")
    for k in model.K:
        # Calcular la distancia total recorrida por el vehículo k
        distancia_total = sum(
            (model.distance[i, j]/1000 if vehicles_dict[k][0] != 'DR' else model.distance_air[i, j]) * model.x[i, j, k].value
            for i in model.I for j in model.I if i != j
        )
        file.write(f"  Vehículo {k}: {distancia_total:.2f} km\n")

    # Escribir pesos totales transportados
    file.write("\nPesos totales transportados:\n")
    for k in model.K:
        file.write(f"  Vehículo {k}: {model.W[k].value:.2f} kg\n")

print(f"Resultados escritos en el archivo: {results_path}")

Resultados escritos en el archivo: results.txt


In [17]:
import folium
from folium.plugins import AntPath
import requests

# Crear un mapa centrado en una ubicación aproximada
centro_lat = deposits_df['Latitude'].mean()
centro_lon = deposits_df['Longitude'].mean()
mapa = folium.Map(location=[centro_lat, centro_lon], zoom_start=12)

# Asignar colores según el tipo de vehículo
tipo_colores = {
    "GS": "red",  # Gasoline cars
    "EV": "blue",  # Electric vehicles
    "DR": "green"  # Drones
}

# Agregar marcadores para depósitos
for depot_id, (id, lon, lat) in deposits_dict.items():
    folium.Marker(
        location=[lat, lon],
        popup=f"Depósito {depot_id}",
        icon=folium.Icon(color="blue", icon="home")
    ).add_to(mapa)

# Agregar marcadores para clientes
for client_id, (product, lon, lat) in clients_dict.items():
    folium.Marker(
        location=[lat, lon],
        popup=f"Cliente {client_id} - Producto {product}",
        icon=folium.Icon(color="green", icon="user")
    ).add_to(mapa)

# Función para calcular ruta real con OSRM
def calcular_ruta_osrm(coord_inicio, coord_fin):
    try:
        url = f"http://router.project-osrm.org/route/v1/driving/{coord_inicio[1]},{coord_inicio[0]};{coord_fin[1]},{coord_fin[0]}"
        params = {"overview": "full", "geometries": "geojson"}
        response = requests.get(url, params=params)
        if response.status_code == 200:
            route = response.json()
            geometry = route["routes"][0]["geometry"]["coordinates"]
            return [(lat, lon) for lon, lat in geometry]
        else:
            print(f"Error OSRM: {response.status_code}")
            return [coord_inicio, coord_fin]
    except Exception as e:
        print(f"Error al calcular ruta OSRM: {e}")
        return [coord_inicio, coord_fin]

# Dibujar los recorridos de los vehículos con estilos según el tipo
for k in model.K:
    tipo_vehiculo = vehicles_dict[k][0]  # Obtener el tipo de vehículo (GS, EV, DR)
    color = tipo_colores[tipo_vehiculo]  # Asignar color según el tipo

    for i in model.I:
        for j in model.I:
            if i != j and model.x[i, j, k].value > 0.5:  # Si hay un recorrido entre i y j
                # Obtener coordenadas de los nodos
                if i in clients_dict:
                    coord_i = (clients_dict[i][2], clients_dict[i][1])  # (lat, lon)
                else:
                    coord_i = (deposits_dict[i][2], deposits_dict[i][1])  # (lat, lon)
                if j in clients_dict:
                    coord_j = (clients_dict[j][2], clients_dict[j][1])  # (lat, lon)
                else:
                    coord_j = (deposits_dict[j][2], deposits_dict[j][1])  # (lat, lon)

                # Calcular la ruta (línea recta para drones, ruta OSRM para otros)
                if tipo_vehiculo == "DR":
                    ruta = [coord_i, coord_j]  # Línea recta
                else:
                    ruta = calcular_ruta_osrm(coord_i, coord_j)  # Camino real

                print(ruta)

                # Dibujar la línea
                AntPath(
                    locations=ruta,
                    color=color,
                    weight=2.5,
                    opacity=1,
                    dash_array=[1, 20],  # Movimiento solo para drones
                    tooltip=f"Vehículo {k} ({tipo_vehiculo}): {i} -> {j}"
                ).add_to(mapa)

# Guardar el mapa en un archivo HTML
mapa.save("recorridos_por_tipo.html")
print("Mapa guardado como 'recorridos_por_tipo.html'")


Error al calcular ruta OSRM: HTTPConnectionPool(host='router.project-osrm.org', port=80): Max retries exceeded with url: /route/v1/driving/-74.11138839002187,4.557320898243896;-74.10991610076434,4.550640992537941?overview=full&geometries=geojson (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000023E70663EB0>: Failed to establish a new connection: [WinError 10060] Se produjo un error durante el intento de conexión ya que la parte conectada no respondió adecuadamente tras un periodo de tiempo, o bien se produjo un error en la conexión establecida ya que el host conectado no ha podido responder'))
[(4.557320898243896, -74.11138839002187), (4.550640992537941, -74.10991610076434)]


KeyboardInterrupt: 